# Linear Regression

Illustration of how to implement a Linear Regeression

In [78]:
# Standard Imports

import torch
from torch import nn as nn
import numpy as np
from torch import tensor, save
import torch.optim as optim
import os

In [79]:
class LinearRegression(nn.Module):
    
    def __init__(self, input_dim):
        
        """
        Initializer for the Model. You should instantiate your layers here.
        This may require you to feed the input and the output dimensions that you require.
        @param input_dim: The dimension of each sample of the input
        """
        
        super(LinearRegression, self).__init__()
        
        self.input_dim = input_dim
        
        # The output dimension of the linear layer is 1 since we need to predict 1 continuous value for each sample
        
        self.linear = nn.Linear(self.input_dim, 1)
    
    def forward(self, x):
        
        """
        In this function, you use the input x and transform it using the layers specified in the
        __init__() function.
        """
        
        linear_transform = self.linear(x)
        
        return linear_transform

In [80]:
def train_linear_regression():
    
    """
    Function that trains the linear model
    """

    # Generating a random tensor with 32 samples each of which is represented through 8 dimensions
    inputs = torch.rand(32, 8)
    
    # Generating the output tensor for illustration.
    labels = torch.rand(32, 1)
    
    # Initializing training specific variables
    model = LinearRegression(input_dim = 8)
    criterion = nn.MSELoss()
    
    # Optimizer to change the weights of the model. Will be explained in next week's class.
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    running_loss = 0.0
    
    epochs = 10
    for ep in range(epochs):
        # In this example, we will use the same input tensor each time, but ideally you should be dividing your
        # data in small minibatches and train one batch in one iteration
        for i in range(0, 100):

            # Set the state of the model to 'train'. This is important for backpropagation step about which you
            # learn in the next class. Backpropagation requires the values computed in the forward() function for
            # each layer. Invoking the .train() function directs the model to save these values for future
            # backpropagation step.
            model.train()
            
            # Very important to reset the gradients to zero before training a minibatch. Otherwise it will keep 
            # adding to gradients computed in the previous iterations
            optimizer.zero_grad()

            # print (inputs, labels)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Perform the backpropagation step to compute gradients
            loss.backward()
            # Perform the gradient update
            optimizer.step()
            
            running_loss += loss.item()
        print('Epoch %d, loss:%.4f' % (ep+1, running_loss/100))
        running_loss = 0.
    
    print('Model State ===>')
    print(model.state_dict())
    
    # Saving the trained model
    save({'epochs': 2, 'model_state_dict': model.state_dict()}, 'linear_regression.ckpt')

In [81]:
def predict_linear_regression():

    # Generating a random tensor with 32 samples each of which is represented through 8 dimensions
    inputs = torch.rand(32, 8)
    
    # Initializing an object of LinearRegression class
    model = LinearRegression(input_dim = 8)
    
    # Load the checkpoint stored from the file
    checkpoint = torch.load('linear_regression.ckpt')
    
    # Load the Model state
    model.load_state_dict(checkpoint['model_state_dict'])
        
    # Set the flag to .eval() since we now don't need to store the values computed in the forward(). This is
    # at the prediction time we don't need to perform a backpropagation step.
    model.eval()
    
    outputs = model(inputs)
    
    # Ignore the detach() for now. We'll discuss that in next section
    return outputs.detach().numpy()

In [82]:
# Call the training function followed by the prediction function
print('Training Linear Regression Model\n')
train_linear_regression()

print('\nPredicting using Linear Regression Model')
outputs = predict_linear_regression()
print(outputs)

Training Linear Regression Model

Epoch 1, loss:0.1311
Epoch 2, loss:0.0536
Epoch 3, loss:0.0530
Epoch 4, loss:0.0529
Epoch 5, loss:0.0528
Epoch 6, loss:0.0528
Epoch 7, loss:0.0528
Epoch 8, loss:0.0528
Epoch 9, loss:0.0528
Epoch 10, loss:0.0528
Model State ===>
OrderedDict([('linear.weight', tensor([[ 0.2297,  0.0362,  0.4637,  0.0807,  0.1550, -0.1666,  0.2378,  0.2464]])), ('linear.bias', tensor([-0.0522]))])

Predicting using Linear Regression Model
[[0.65844446]
 [0.7625149 ]
 [0.8820373 ]
 [0.40817782]
 [0.539098  ]
 [0.49190396]
 [0.5486369 ]
 [0.3294576 ]
 [0.26806343]
 [0.9181377 ]
 [0.5740857 ]
 [0.40673345]
 [0.73572147]
 [0.42749745]
 [0.5737763 ]
 [0.5658042 ]
 [0.5307116 ]
 [0.72412723]
 [0.6921247 ]
 [0.70571715]
 [0.5919994 ]
 [0.32175344]
 [0.47406918]
 [0.73307294]
 [0.64957833]
 [0.6208963 ]
 [0.5455258 ]
 [0.95511454]
 [0.5975401 ]
 [0.60348624]
 [0.87858635]
 [0.79118305]]


# Logistic Classifier

Illustration of how to implement a Logistic Classifier

In [83]:
class LogisticClassifier(nn.Module):
    
    def __init__(self, input_dim):
        
        """
        Initializer for the Model. You should instantiate your layers here.
        This may require you to feed the input and the output dimensions that you require.
        @param input_dim: The dimension of each sample of the input
        """
        
        super(LogisticClassifier , self).__init__()
        
        self.input_dim = input_dim
        
        # The output dimension of the linear layer is 1 since we are using a sigmoid layer to
        # to perform binary classification
        
        self.linear = nn.Linear(self.input_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        
        """
        In this function, you use the input x and transform it using the layers specified in the
        __init__() function.
        """
        
        linear_transform = self.linear(x)
        sigmoid = self.sigmoid(linear_transform)
        
        return linear_transform, sigmoid

In [84]:
def train_logistic_classifier():
    
    """
    Function that trains the linear model
    """

    # Generating a random tensor with 32 samples each of which is represented through 8 dimensions
    inputs = torch.rand(32, 8)
    
    # Generating the output tensor for illustration.
    labels = torch.randint(low=0, high=2, size=(32, 1))
    
    # Initializing training specific variables
    model = LogisticClassifier(input_dim = 8)
    
    # Refer http://www.philkr.net/cs342/doc/sigmoid/ for more discussion on this loss function.
    criterion = nn.BCEWithLogitsLoss()
    
    # Optimizer to change the weights of the model. Will be explained in next week's class.
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    running_loss = 0.0
    
    epochs = 10
    for ep in range(epochs):
        # In this example, we will use the same input tensor each time, but ideally you should be dividing your
        # data in small minibatches and train one batch in one iteration
        for i in range(0, 100):

            # Set the state of the model to 'train'. This is important for backpropagation step about which you
            # learn in the next class. Backpropagation requires the values computed in the forward() function for
            # each layer. Invoking the .train() function directs the model to save these values for future
            # backpropagation step.
            # Refer https://pytorch.org/docs/stable/nn.html#torch.nn.Module.train
            model.train()
            
            # Very important to reset the gradients to zero before training a minibatch. Otherwise it will keep 
            # adding to gradients computed in the previous iterations
            # Refer https://pytorch.org/docs/stable/nn.html#torch.nn.Module.zero_grad
            optimizer.zero_grad()

            # print (inputs, labels)

            # During training, since we are using the BCEWithLogitsLoss which is a combination of a sigmoid layer
            # and binary cross entropy, hence, we don't need to use the output of the sigmoid layer from the model.
            # Thus, just ignore for this step.
            
            outputs, _ = model(inputs)
            loss = criterion(outputs, labels)
            
            # Perform the backpropagation step to compute gradients
            loss.backward()
            # Perform the gradient update
            optimizer.step()
            
            running_loss += loss.item()
        print('Epoch %d, loss:%.4f' % (ep+1, running_loss/100))
        running_loss = 0.
    
    print('Model State ===>')
    print(model.state_dict())
    
    # Saving the trained model
    save({'epochs': 2, 'model_state_dict': model.state_dict()}, 'logistic_classifier.ckpt')

In [85]:
def predict_binary_class():

    # Generating a random tensor with 32 samples each of which is represented through 8 dimensions
    inputs = torch.rand(32, 8)
    
    # Initializing an object of LogisticClassifier Class
    model = LogisticClassifier(input_dim = 8)
    
    # Load the checkpoint stored from the file
    checkpoint = torch.load('logistic_classifier.ckpt')
    
    # Load the Model state
    model.load_state_dict(checkpoint['model_state_dict'])
        
    # Set the flag to .eval() since we now don't need to store the values computed in the forward(). This is
    # at the prediction time we don't need to perform a backpropagation step.
    # Refer https://pytorch.org/docs/stable/nn.html#torch.nn.Module.eval
    model.eval()
    
    # We don't need the output of the linear transformation for the prediction. So we just ignore that
    _, outputs = model(inputs)
    
    # Perform the boundary test to classify as 0 or 1
    outputs = outputs >= 0.5
    
    # Ignore the detach() for now. We'll discuss that in next section
    return outputs.detach().numpy()

In [87]:
# Call the training function followed by the prediction function
print('Training Logistic Classifier Model\n') 
train_logistic_classifier()

print('\nPrediciting using Logisitc Classifier\n')
outputs = predict_binary_class()
print(outputs)

Training Logistic Classifier Model

Epoch 1, loss:0.6511
Epoch 2, loss:0.6168
Epoch 3, loss:0.5962
Epoch 4, loss:0.5806
Epoch 5, loss:0.5684
Epoch 6, loss:0.5588
Epoch 7, loss:0.5509
Epoch 8, loss:0.5444
Epoch 9, loss:0.5389
Epoch 10, loss:0.5343
Model State ===>
OrderedDict([('linear.weight', tensor([[ 1.0220, -1.7958, -1.0330, -1.1648, -1.8942,  0.0168,  1.0348,  0.8753]])), ('linear.bias', tensor([0.8121]))])

Prediciting using Logisitc Classifier

[[1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]]
